In [1]:
import os

In [2]:
%pwd

'/Users/adityakumar/BankCD-Prediction/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/adityakumar/BankCD-Prediction'

Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [7]:
from src.bankcdProject.constant import *
from src.bankcdProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBoost
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [10]:
import os
import pandas as pd
from urllib.parse import urlparse
import numpy as np
import joblib
from sklearn.metrics import roc_auc_score, accuracy_score

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self, y_test, y_test_pred, y_test_proba):
        accuracy = accuracy_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, y_test_proba)
        
        return accuracy, roc_auc

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        y_test_pred = model.predict(test_x)
        y_test_proba = model.predict_proba(test_x)[:, 1]

        accuracy, roc_auc = self.eval_metrics(test_y, y_test_pred, y_test_proba)
        
        # Saving metrics as local
        scores = {"accuracy": accuracy, "roc_auc": roc_auc}
        save_json(path=Path(self.config.metric_file_name), data=scores)

Pipeline

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-11-13 21:57:44,658: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-13 21:57:44,670: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-13 21:57:44,673: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-13 21:57:44,675: INFO: common: created directory at: artifacts]
[2024-11-13 21:57:44,675: INFO: common: created directory at: artifacts/model_evaluation]
[2024-11-13 21:57:45,018: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
